## Exercício 2 (Entrega em 04/05/2021)

> Nome: Michelle Wingter da Silva
>
> NUSP: 10783243

### Configurando ambiente

Importando bibliotecas necessárias

In [1]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np

Inicializando janela

In [2]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)
window = glfw.create_window(720, 720, "Exercicio 1", None, None)
glfw.make_context_current(window)

Capturando eventos de teclado e mouse

In [3]:
def key_event(window,key,scancode,action,mods):
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    
glfw.set_key_callback(window,key_event)

def mouse_event(window,button,action,mods):
    print('[mouse event] button=',button)
    print('[mouse event] action=',action)
    print('[mouse event] mods=',mods)
    print('-------')
glfw.set_mouse_button_callback(window,mouse_event)

GLSL para Vertex Shader

In [4]:
vertex_code = """
        attribute vec2 position;
        void main(){
            gl_Position = vec4(position,0.0,1.0);
        }
        """

GLSL para Fragment Shader

In [5]:
fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [6]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

Associando nosso código-fonte aos slots solicitados

In [7]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

Compilando o Vertex Shader

In [8]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

Compilando o Fragment Shader

In [9]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

Associando os programas compilado ao programa principal

In [10]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

Linkagem do programa

In [11]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

In [12]:
# preparando espaço para 3 vértices usando 2 coordenadas (x,y)
vertices = np.zeros(19, [("position", np.float32, 2)])

In [13]:
# preenchendo as coordenadas de cada vértice
vertices['position'] = [
                            ( 0.0, 0.0), # vertice 0 - ponto
                            (+0.5,+0.5), # vertice 1 - ponto
                            (-0.5,-0.5), # vertice 2 - ponto
    
                            ( 0.0,+1.0), # vertice 3 - linha
                            (+0.0,-1.0), # vertice 4 - linha
                            ( 1.0, 0.0), # vertice 5 - linha
                            (-1.0, 0.0), # vertice 6 - linha
                            ( 0.0, 0.0), # vertice 7 - linha
                            (-0.5,-0.5), # vertice 8 - linha
    
                            ( 0.0, 0.0), # vertice 9 - triangulo
                            (+0.5,+1.0), # vertice 10 - triangulo
                            (+1.0, 0.0), # vertice 11 - triangulo
                            (-1.0,-1.0), # vertice 12 - triangulo
                            (-0.5, 0.0), # vertice 13 - triangulo
                            (+0.0,-1.0), # vertice 14 - triangulo
    
                            (-1.0, +1.0), # vertice 15 - quadrado
                            (-1.0, +0.5), # vertice 16 - quadrado
                            (-0.5, +1.0), # vertice 17 - quadrado
                            (-0.5, +0.5), # vertice 18 - quadrado
    
                            # vertices 19 a 19+64 - circulo
                        ]

In [14]:
# preparando espaço para adicionar os vértices do circulo
vertices.resize(19+65)

In [15]:
import math # para calculo com sin e cos

num_vertices = 64 # define a "qualidade" do circulo
pi = 3.14
counter = 0
radius = 0.2
angle = 0.0
centerX = 0.5
centerY = - 0.5
for counter in range(num_vertices):
    angle += 2*pi/num_vertices 
    x = math.cos(angle)*radius
    y = math.sin(angle)*radius
    vertices[counter+19] = [x+centerX,y+centerY]

Para enviar nossos dados da CPU para a GPU, precisamos requisitar um slot.

In [16]:
# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

Abaixo, nós enviamos todo o conteúdo da variável vertices.

In [17]:
# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)

Associando variáveis do programa GLSL (Vertex Shaders) com nossos dados

In [18]:
# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

In [19]:
loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)

In [20]:
glVertexAttribPointer(loc, 2, GL_FLOAT, False, stride, offset)

### Vamos pegar a localização da variável color (uniform) para que possamos alterá-la em nosso laço da janela!

In [21]:
loc_color = glGetUniformLocation(program, "color")
R = 1.0
G = 0.0
B = 0.0

Nesse momento, nós exibimos a janela!

In [22]:
glfw.show_window(window)

#### Funções de desenho

In [23]:
def desenha_ponto(inicio, num):
    glDrawArrays(GL_POINTS, inicio, num) #pontos

def desenha_linha(inicio, num):
     glDrawArrays(GL_LINES, inicio, num) #linhas
        
def desenha_triangulo(inicio, num):
     glDrawArrays(GL_TRIANGLES, inicio, num) #linhas
        
def desenha_quadrado(inicio, num):
     glDrawArrays(GL_TRIANGLE_STRIP, inicio, num) #linhas
        
def desenha_circulo(inicio, num):
     glDrawArrays(GL_TRIANGLE_FAN, inicio, num) #linhas
        
def pinta(r, g, b):
     glUniform4f(loc_color, r, g, b, 1.0) ### modificando a cor do objeto!

### Loop principal da janela.

In [24]:
while not glfw.window_should_close(window):

    glfw.poll_events() 

    
    glClear(GL_COLOR_BUFFER_BIT) 
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    #glPolygonMode(GL_FRONT_AND_BACK, GL_LINE)
    '''
    glUniform4f(loc_color, 1.0, 0, 0, 1.0) ### modificando a cor do objeto!
    
    ### glDrawArrays
    glDrawArrays(GL_POINTS, 0, 3) #pontos
    
    glUniform4f(loc_color, 0, 0, 0, 1.0) ### modificando a cor do objeto!
    glDrawArrays(GL_LINES, 3, 6) #linhas
    
    glUniform4f(loc_color, 1.0, 0, 0, 1.0) ### modificando a cor do objeto!
    glDrawArrays(GL_TRIANGLES, 9, 6) #triangulos
    
    glUniform4f(loc_color, 1.0, 1.0, 0, 1.0) ### modificando a cor do objeto!
    glDrawArrays(GL_TRIANGLE_STRIP, 15, 4) #quadrados
    
    glUniform4f(loc_color, 1.0, 0, 1.0, 1.0) ### modificando a cor do objeto!
    glDrawArrays(GL_TRIANGLE_FAN, 19, 64) #circulo
    '''
    
    pinta(1,0,0)
    desenha_ponto(0,3)
    
    pinta(0,0,0)
    desenha_linha(3,6)
    
    pinta(1,0,0)
    desenha_triangulo(9,6)
    
    pinta(1,1,0)
    desenha_quadrado(15,4)
    
    pinta(1,0,1)
    desenha_circulo(19,64)

    glfw.swap_buffers(window)

glfw.terminate()

## Resultado:

![imagem](exercicio1.PNG)